In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

In [2]:
import os 
os.chdir('D:/university/biobigdata/5-breast cancer survival analysis/5-breast cancer survival analysis')

In [3]:
# 读取数据
data = pd.read_csv('D:/university/biobigdata/5-breast cancer survival analysis/5-breast cancer survival analysis/clean_data2.csv')

# 将数据集分为训练集和测试集
train_data, test_data = train_test_split(data, test_size=0.2, random_state=123)

In [4]:
data

,death_from_cancer,patient_id,age_at_diagnosis,type_of_breast_surgery,cancer_type_detailed,cellularity,chemotherapy,pam50_._claudin.low_subtype,cohort,er_status_measured_by_ihc,...,mtap_mut,ppp2cb_mut,smarcd1_mut,nras_mut,ndfip1_mut,hras_mut,prps2_mut,smarcb1_mut,stmn2_mut,siah1_mut
0,0,2,43.19,2,3,2,0,4,1,3,...,0,0,0,0,0,0,0,0,0,0
1,1,5,48.87,3,3,2,1,5,1,3,...,0,0,0,0,0,0,0,0,0,0
2,0,6,47.68,3,6,4,1,5,1,3,...,0,0,0,0,0,0,0,0,0,0
3,1,8,76.97,3,6,2,1,5,1,3,...,0,0,0,0,0,0,0,0,0,0
4,1,10,78.77,3,3,4,0,5,1,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,0,6233,70.65,2,6,2,0,5,5,3,...,0,0,0,0,0,0,0,0,0,0
1306,1,6237,75.62,3,3,2,0,1,5,2,...,0,0,0,0,0,0,0,0,0,0
1307,0,6239,52.84,2,6,2,1,7,5,3,...,0,0,0,0,0,0,0,0,0,0
1308,1,6251,48.59,3,3,2,1,1,5,2,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# data.iloc[:, 1:30]
data.iloc[:,518]
data.iloc[:,27]

0       1
1       2
2       2
3       2
4       4
       ..
1305    1
1306    1
1307    2
1308    2
1309    2
Name: tumor_stage, Length: 1310, dtype: int64

In [7]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.x = data.iloc[:, 2:].values # 所有列，不要patient_id
#         self.x = data.iloc[:, 2:518].values #去除突变情况的列
#         self.x = data.iloc[:, 2:28].values
        self.y = data.iloc[:, 0].values
        self.mean = self.x.mean(axis=0)
        self.std = self.x.std(axis=0)
        self.x = (self.x - self.mean) / (self.std+1e-8)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        return torch.from_numpy(self.x[index]).float(), torch.tensor(self.y[index]).float()


In [8]:
train_set = CustomDataset(train_data)
test_set = CustomDataset(test_data)
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


In [9]:
len(train_set[0][0])
# train_set[0][0]

688

In [10]:
# 构建全连接神经网络模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(688, 64)
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x=x.view(-1,690)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x
    

In [11]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # 定义三个全连接层，输入层有 150*150=22500 个节点，输出层有 14 个节点
        self.fc1 = torch.nn.Linear(688, 300) #所有列
#         self.fc1 = torch.nn.Linear(516, 300) #去除突变情况的列
#         self.fc1 = torch.nn.Linear(26, 300)
        self.fc2 = torch.nn.Linear(300, 100)
        self.fc3 = torch.nn.Linear(100, 1)
        # 定义两个激活函数层，使用 ReLU 函数
        self.relu1 = torch.nn.ReLU()
        self.relu2 = torch.nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # 前向传播过程，将输入张量 x 展平为一维向量
        x = x.view(-1, 688) #所有列
#         x = x.view(-1, 516) #去除突变情况的列
#         x = x.view(-1, 26)
        # 依次通过全连接层和激活函数层
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        # 返回输出张量 x
        return x

In [12]:
# 训练模型
model = MLP()
# model = Net()

# 定义损失函数和优化器
criterion = nn.BCELoss()
# criterion = nn.BCEWithLogitsLoss()
# criterion = nn.CrossEntropyLoss() # 不适用
# optimizer = optim.Adam(model.parameters(), lr=0.01)

weight_decay = 0.001
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=weight_decay)

In [13]:
# 测试能否训练

# running_loss = 0
# for x, y in train_loader:
#     optimizer.zero_grad()
#     output = model(x)
#     print(output.shape)
# #     print(output)
# #     print(y.shape)
# #     print(y)
#     output = output.reshape(-1)
# #     print(output)
#     loss = criterion(output, y)
#     loss.backward()
#     optimizer.step()
#     running_loss += loss.item() * len(y)
# print('epoch: %d, loss: %.3f' % (epoch + 1, running_loss / len(train_set)))

In [14]:
for epoch in range(50):
    running_loss = 0
    for x, y in train_loader:
        optimizer.zero_grad()
        output = model(x)
        output = output.reshape(-1)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * len(y)
    print('epoch: %d, loss: %.3f' % (epoch + 1, running_loss / len(train_set)))
    # 在测试集上进行验证
    total_loss = 0
    total_acc = 0
    model.eval()
    with torch.no_grad():
        for x, y in test_loader:
            output = model(x)
            output = output.reshape(-1)
            total_loss += criterion(output, y).item() * len(y)
            total_acc += ((output > 0.5).float() == y).float().sum().item() / len(y)
    avg_loss = total_loss / len(test_set)
    avg_acc = total_acc / len(test_loader)
    print('test_loss: %.3f, test_accuracy: %.3f' % (avg_loss, avg_acc))

# # 在测试集上进行验证
# total_loss = 0
# total_acc = 0
# model.eval()
# with torch.no_grad():
#     for x, y in test_loader:
#         output = model(x)
#         output = output.reshape(-1)
#         total_loss += criterion(output, y).item() * len(y)
#         total_acc += ((output > 0.5).float() == y).float().sum().item() / len(y)
# avg_loss = total_loss / len(test_set)
# avg_acc = total_acc / len(test_loader)
# print('test_loss: %.3f, test_accuracy: %.3f' % (avg_loss, avg_acc))

epoch: 1, loss: 0.675
test_loss: 0.656, test_accuracy: 0.609
epoch: 2, loss: 0.521
test_loss: 0.752, test_accuracy: 0.634
epoch: 3, loss: 0.430
test_loss: 0.639, test_accuracy: 0.678
epoch: 4, loss: 0.330
test_loss: 0.763, test_accuracy: 0.661
epoch: 5, loss: 0.286
test_loss: 1.063, test_accuracy: 0.574
epoch: 6, loss: 0.193
test_loss: 1.454, test_accuracy: 0.609
epoch: 7, loss: 0.191
test_loss: 1.139, test_accuracy: 0.628
epoch: 8, loss: 0.195
test_loss: 1.442, test_accuracy: 0.612
epoch: 9, loss: 0.123
test_loss: 1.581, test_accuracy: 0.596
epoch: 10, loss: 0.108
test_loss: 1.676, test_accuracy: 0.608
epoch: 11, loss: 0.131
test_loss: 1.671, test_accuracy: 0.582
epoch: 12, loss: 0.142
test_loss: 1.407, test_accuracy: 0.594
epoch: 13, loss: 0.082
test_loss: 1.770, test_accuracy: 0.630
epoch: 14, loss: 0.047
test_loss: 2.010, test_accuracy: 0.558
epoch: 15, loss: 0.152
test_loss: 1.910, test_accuracy: 0.590
epoch: 16, loss: 0.213
test_loss: 1.984, test_accuracy: 0.597
epoch: 17, loss: 

In [15]:
# 加入主成分分析

import pandas as pd
from sklearn.decomposition import PCA

# 返回一个元组，第一项是对mRNA信息进行PCA后的data（删除原来列，PCA列添加在后面）
# 第二项是PCA各个主成分中对应mRNA的占比
# 使用预先处理过（删去了缺失值，overallsurvival，death位置移动到表格第一列并且改为只有0和1）的"clean_data2.csv"
def PCA_mRNA(num_components, data = pd.read_csv("clean_data2.csv")):

    data = pd.read_csv("clean_data2.csv")
    mRNAs = data.loc[:,"brca1":"ugt2b7"]
    print(mRNAs.head())

    num_miss = mRNAs.isnull().sum()
    print(f"Number of missing values: {sum(num_miss)}")

    pca = PCA(n_components=num_components)
    principle_components = pca.fit_transform(mRNAs)
    print(f"Shape of principle components: {principle_components.shape}")

    original_features = pca.components_
    print(f"Principle components -- Original features:\n{original_features}")

    data_pca = data.drop(columns=data.loc[:,"brca1":"ugt2b7"].columns)
    for i in range(num_components):
        data_pca[f"Principle_Component_{i+1}"] = principle_components[:,i]

    return data_pca, original_features

In [16]:
data_ori = pd.read_csv('D:/university/biobigdata/5-breast cancer survival analysis/5-breast cancer survival analysis/clean_data2.csv')
data2,feature = PCA_mRNA(50,data_ori)

    brca1   brca2   palb2    pten    tp53     atm    cdh1   chek2     nbn  \
0 -1.3800  0.2777 -1.2154  0.5296 -0.0136 -0.2659  1.3594  0.7961  0.5419   
1  0.0670 -0.8426  0.2114 -0.3326  0.5141 -0.0803  1.1398  0.4187 -0.4030   
2  0.6744 -0.5428 -1.6592  0.6369  1.6708 -0.8880  1.2491 -1.1889 -0.4174   
3  1.2932 -0.9039 -0.7219  0.2168  0.3484  0.3897  0.9131  0.9356  0.7675   
4 -0.4341  0.6931  0.6909  1.0840 -1.9371  0.9252  1.1520  0.7951 -0.9677   

      nf1  ...  srd5a1  srd5a2  srd5a3     st7    star    tnk2   tulp4  \
0 -2.6059  ... -0.4412  0.4534  0.4068  0.7634  0.0231  0.9121 -0.9538   
1 -1.1305  ... -0.5381  0.0668  0.8344  1.7227  0.4024 -3.7172 -1.5538   
2 -0.6165  ... -0.5630 -0.7078  0.8228  0.6819 -0.1948 -2.3286 -0.9924   
3 -0.2940  ... -0.5845 -0.3544 -1.0150  2.2961  0.1817 -0.1572  0.0427   
4 -0.7750  ... -0.3910  0.0395  0.8191  0.1977 -0.2302 -0.5092 -0.3021   

   ugt2b15  ugt2b17  ugt2b7  
0  -0.2264   0.5398 -0.8920  
1   1.3701  -0.1078  0.3655  
2 

In [17]:
data2

,death_from_cancer,patient_id,age_at_diagnosis,type_of_breast_surgery,cancer_type_detailed,cellularity,chemotherapy,pam50_._claudin.low_subtype,cohort,er_status_measured_by_ihc,...,Principle_Component_41,Principle_Component_42,Principle_Component_43,Principle_Component_44,Principle_Component_45,Principle_Component_46,Principle_Component_47,Principle_Component_48,Principle_Component_49,Principle_Component_50
0,0,2,43.19,2,3,2,0,4,1,3,...,-0.936911,0.814736,1.867345,1.658807,-1.863991,1.095932,1.370988,0.600619,-3.477923,-1.944872
1,1,5,48.87,3,3,2,1,5,1,3,...,-0.254122,-2.830932,1.296693,0.129446,-0.150905,0.584752,0.499728,-1.666691,-0.826845,-0.313810
2,0,6,47.68,3,6,4,1,5,1,3,...,-0.010624,-0.253833,4.314712,-0.641174,0.224906,-0.827866,-0.543045,-1.739714,1.327241,-1.607571
3,1,8,76.97,3,6,2,1,5,1,3,...,-3.363583,1.101015,-3.727630,0.875414,3.745942,0.780397,-1.991749,-0.202281,2.435188,-1.076689
4,1,10,78.77,3,3,4,0,5,1,3,...,-0.588511,0.908041,-0.214866,2.298804,2.196755,-0.771571,2.435560,0.563607,0.130661,-0.579504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,0,6233,70.65,2,6,2,0,5,5,3,...,0.372999,0.077146,-0.239522,0.094094,-0.889182,-1.426109,1.788790,-0.116529,-0.894413,1.176954
1306,1,6237,75.62,3,3,2,0,1,5,2,...,-2.611736,-2.858031,1.209614,-0.945801,-1.614824,-1.302237,1.486283,0.242835,1.080865,-1.015153
1307,0,6239,52.84,2,6,2,1,7,5,3,...,-0.210152,0.127777,-0.232353,0.968256,-1.650388,1.250320,0.180820,-2.439122,0.441873,0.168700
1308,1,6251,48.59,3,3,2,1,1,5,2,...,-1.739718,-2.802831,2.334958,1.398177,1.235989,0.934626,0.968225,-4.283895,1.027530,-1.706979


In [18]:
class CustomDataset(Dataset):
    def __init__(self, data):
#         self.x = data.iloc[:, 2:518].values
        self.x = data.iloc[:, 2:].values
        self.y = data.iloc[:, 0].values
        self.mean = self.x.mean(axis=0)
        self.std = self.x.std(axis=0)
        self.x = (self.x - self.mean) / (self.std+1e-8)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        return torch.from_numpy(self.x[index]).float(), torch.tensor(self.y[index]).float()

In [19]:
train_data, test_data = train_test_split(data2, test_size=0.2, random_state=123)
train_set = CustomDataset(train_data)
test_set = CustomDataset(test_data)
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)
len(train_set[0][0])
# train_set[0][0]

249

In [20]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # 定义三个全连接层，输入层有 150*150=22500 个节点，输出层有 14 个节点
#         self.fc1 = torch.nn.Linear(689, 300)
#         self.fc1 = torch.nn.Linear(516, 300)
        self.fc1 = torch.nn.Linear(249, 300)
        self.fc2 = torch.nn.Linear(300, 100)
        self.fc3 = torch.nn.Linear(100, 1)
        # 定义两个激活函数层，使用 ReLU 函数
        self.relu1 = torch.nn.ReLU()
        self.relu2 = torch.nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # 前向传播过程，将输入张量 x 展平为一维向量
#         x = x.view(-1, 689)
#         x = x.view(-1, 516)
        x = x.view(-1, 249)
        # 依次通过全连接层和激活函数层
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        # 返回输出张量 x
        return x

In [21]:
# 训练模型
model = MLP()
# model = Net()

# 定义损失函数和优化器
criterion = nn.BCELoss()
# criterion = nn.BCEWithLogitsLoss()
# criterion = nn.CrossEntropyLoss() # 不适用
# optimizer = optim.Adam(model.parameters(), lr=0.01)

weight_decay = 0.001
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=weight_decay)

In [22]:
for epoch in range(50):
    running_loss = 0
    for x, y in train_loader:
        optimizer.zero_grad()
        output = model(x)
        output = output.reshape(-1)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * len(y)
    print('epoch: %d, loss: %.3f' % (epoch + 1, running_loss / len(train_set)))
    # 在测试集上进行验证
    total_loss = 0
    total_acc = 0
    model.eval()
    with torch.no_grad():
        for x, y in test_loader:
            output = model(x)
            output = output.reshape(-1)
            total_loss += criterion(output, y).item() * len(y)
            total_acc += ((output > 0.5).float() == y).float().sum().item() / len(y)
    avg_loss = total_loss / len(test_set)
    avg_acc = total_acc / len(test_loader)
    print('test_loss: %.3f, test_accuracy: %.3f' % (avg_loss, avg_acc))

epoch: 1, loss: 0.637
test_loss: 0.807, test_accuracy: 0.637
epoch: 2, loss: 0.506
test_loss: 0.796, test_accuracy: 0.616
epoch: 3, loss: 0.426
test_loss: 0.872, test_accuracy: 0.647
epoch: 4, loss: 0.356
test_loss: 0.884, test_accuracy: 0.616
epoch: 5, loss: 0.324
test_loss: 0.930, test_accuracy: 0.616
epoch: 6, loss: 0.245
test_loss: 1.215, test_accuracy: 0.604
epoch: 7, loss: 0.213
test_loss: 2.116, test_accuracy: 0.587
epoch: 8, loss: 0.168
test_loss: 2.149, test_accuracy: 0.630
epoch: 9, loss: 0.165
test_loss: 1.854, test_accuracy: 0.630
epoch: 10, loss: 0.154
test_loss: 2.408, test_accuracy: 0.633
epoch: 11, loss: 0.117
test_loss: 3.178, test_accuracy: 0.639
epoch: 12, loss: 0.129
test_loss: 1.854, test_accuracy: 0.609
epoch: 13, loss: 0.118
test_loss: 3.026, test_accuracy: 0.637
epoch: 14, loss: 0.120
test_loss: 5.213, test_accuracy: 0.633
epoch: 15, loss: 0.166
test_loss: 3.189, test_accuracy: 0.615
epoch: 16, loss: 0.118
test_loss: 4.339, test_accuracy: 0.644
epoch: 17, loss: 